# Analyse de Sentiment avec Chiffrement Homomorphique Complet (FHE)

Ce notebook contient l'ensemble du projet pour l'analyse de sentiment avec FHE.

**Etapes:**
1. Installation des dependances
2. Definition de la classe TextProcessor
3. Definition des utilitaires du modele
4. Chargement et preparation des donnees
5. Entrainement du modele
6. Compilation pour FHE
7. Test des predictions FHE
8. Demonstration interactive


## Etape 1: Installation des Dependances


In [ ]:
# Installation de tous les packages requis
# Cette etape peut prendre quelques minutes
%pip install -q concrete-ml transformers datasets torch scikit-learn xgboost gradio numpy pandas tqdm

print("Installation terminee!")


## Etape 2: Import des Bibliotheques


In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from typing import List
import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from datasets import load_dataset
import time
import warnings
import pickle
import os
from pathlib import Path
from concrete.ml.sklearn import XGBClassifier
from concrete.ml.deployment import FHEModelDev
import gradio as gr

warnings.filterwarnings('ignore')

print("Toutes les bibliotheques ont ete importees avec succes!")


## Etape 3: Classe TextProcessor

Cette classe convertit le texte en representations vectorielles utilisables par le modele FHE.


In [ ]:
class TextProcessor:
    """Classe pour transformer le texte en representations vectorielles avec RoBERTa."""
    
    def __init__(self, model_name: str = "cardiffnlp/twitter-roberta-base-sentiment-latest", device: str = None):
        """Initialise le processeur de texte."""
        if device is None:
            self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
            
        print(f"Chargement du modele {model_name} sur {self.device}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model = self.model.to(self.device)
        self.model.eval()
        
    def text_to_tensor(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """
        TRANSFORMATION TEXTE EN VECTEUR NUMERIQUE (EN CLAIR)
        
        Convertit le texte en representation vectorielle de 768 dimensions avec RoBERTa.
        Cette etape est EN CLAIR (avant le chiffrement FHE).
        
        ETAPES:
        1. TOKENIZATION: Le texte est divise en tokens (mots/sous-mots)
        2. ENCODING: Les tokens sont convertis en IDs numeriques
        3. ROERTA: Passage a travers le modele RoBERTa pre-entraine
        4. EXTRACTION: Recuperation des etats caches (representations internes)
        5. POOLING: Moyenne des representations pour obtenir un vecteur par texte
        
        RESULTAT:
        - Format: Array numpy (n_texts, 768)
        - Type: float32 (valeurs reelles)
        - Dimensions: 768 (taille de la couche cachee de RoBERTa)
        
        IMPORTANT: Cette etape est EN CLAIR (avant le chiffrement FHE)
        """
        if isinstance(texts, str):
            texts = [texts]
            
        # ETAPE 1: TOKENIZATION
        tokenized_texts = [
            self.tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=512)
            for text in texts
        ]
        
        output_hidden_states_list = []
        
        # ETAPE 2: TRAITEMENT PAR BATCH (pour l'efficacite)
        for i in tqdm.tqdm(range(0, len(tokenized_texts), batch_size), desc="Traitement des textes"):
            batch = tokenized_texts[i:i + batch_size]
            
            # ETAPE 3: PADDING (alignement des longueurs)
            max_len = max(t.shape[1] for t in batch)
            batch_tensors = []
            
            for tokens in batch:
                if tokens.shape[1] < max_len:
                    padding = torch.zeros(1, max_len - tokens.shape[1], dtype=tokens.dtype)
                    tokens = torch.cat([tokens, padding], dim=1)
                batch_tensors.append(tokens)
            
            batch_tensor = torch.cat(batch_tensors, dim=0).to(self.device)
            
            # ETAPE 4: PASSAGE A TRAVERS ROERTA
            with torch.no_grad():
                outputs = self.model(batch_tensor, output_hidden_states=True)
                hidden_states = outputs.hidden_states[-1]
                # ETAPE 5: POOLING (moyenne sur les tokens)
                text_representations = hidden_states.mean(dim=1)
                text_representations = text_representations.cpu().numpy()
                
            output_hidden_states_list.append(text_representations)
        
        # RESULTAT FINAL: (n_texts, 768) en float32
        # RAPPEL: Ces donnees sont EN CLAIR, elles seront chiffrees plus tard
        return np.concatenate(output_hidden_states_list, axis=0)


## Etape 4: Utilitaires du Modele

Fonctions pour compiler, sauvegarder et charger le modele FHE.


In [ ]:
def compile_model(model: XGBClassifier, X_sample):
    """
    ETAPE CRITIQUE: Compiler le modele pour l'execution FHE.
    
    Cette fonction transforme le modele XGBoost en circuit cryptographique FHE.
    
    CE QUI SE PASSE:
    1. QUANTIFICATION: Conversion float → entiers (necessaire pour FHE)
       - Les valeurs float sont arrondies et converties en entiers
       - La precision est reduite (n_bits, typiquement 2-3 bits)
       - Exemple: 0.75 → 3 (si n_bits=2, on a 4 valeurs possibles: 0,1,2,3)
    
    2. COMPILATION: Transformation du modele en circuit FHE
       - Le modele XGBoost est converti en operations cryptographiques
       - Chaque operation (addition, multiplication) devient une operation FHE
       - Generation des cles cryptographiques (cle secrete, cle publique, cles d'evaluation)
    
    3. CIRCUIT FHE: Creation d'un circuit executable sur donnees chiffrees
       - Le circuit peut traiter des ciphertexts (donnees chiffrees)
       - Les resultats sont aussi des ciphertexts
    """
    print("Compilation du modele pour FHE (cela peut prendre quelques minutes)...")
    # ICI: Quantification + Compilation en circuit FHE
    # Concrete-ML fait automatiquement:
    # - Quantification des donnees X_sample (float → int)
    # - Transformation du modele en circuit FHE
    # - Generation des cles cryptographiques
    model.compile(X_sample)
    print("Compilation terminee!")
    return model


def save_model(model: XGBClassifier, model_name: str = "sentiment_fhe_model"):
    """
    Sauvegarde le modele FHE compile.
    
    Sauvegarde le modele avec:
    - Le circuit FHE compile
    - Les cles cryptographiques (cle publique, cles d'evaluation)
    - Les parametres de quantification
    """
    os.makedirs("models", exist_ok=True)
    
    # Sauvegarde du modele FHE avec toutes ses cles
    # FHEModelDev encapsule le modele compile + les cles cryptographiques
    fhe_api = FHEModelDev(model_name, model)
    # Sauvegarde dans models/sentiment_fhe_model/:
    # - Le circuit FHE compile
    # - Les cles cryptographiques (necessaires pour chiffrement/dechiffrement)
    # - Les parametres de quantification
    fhe_api.save("models/")
    
    # Sauvegarder aussi le modele en clair pour reference (optionnel)
    with open(f"models/{model_name}_clear.pkl", "wb") as f:
        pickle.dump(model, f)
    
    print(f"Modele sauvegarde dans models/{model_name}/")


def load_model(model_name: str = "sentiment_fhe_model"):
    """
    Charge un modele FHE sauvegarde.
    
    Args:
        model_name: Nom du modele a charger
        
    Returns:
        Instance FHEModelDev chargee
    """
    model_path = Path("models") / model_name
    
    if not model_path.exists():
        raise FileNotFoundError(f"Modele non trouve: {model_path}")
    
    fhe_api = FHEModelDev(model_name, path=str(model_path.parent))
    return fhe_api


## Etape 5: Chargement et Preparation des Donnees

Chargement du dataset Amazon Polarity (100 exemples pour la rapidite: 80 train, 20 test).


In [ ]:
# Chargement du dataset (100 exemples pour la rapidite: 80 train, 20 test)
print("Chargement du dataset...")
dataset = load_dataset("amazon_polarity", split="train[:100]")
df = pd.DataFrame(dataset)
df = df.rename(columns={"content": "text", "label": "sentiment"})

print(f"Dataset charge: {len(df)} exemples")
print(f"Distribution des sentiments:\n{df['sentiment'].value_counts()}")

# Preparation des donnees
text_X = df['text'].tolist()
y = df['sentiment'].values

# Division train/test (80/20)
text_X_train, text_X_test, y_train, y_test = train_test_split(
    text_X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDonnees d'entrainement: {len(text_X_train)} exemples")
print(f"Donnees de test: {len(text_X_test)} exemples")


## Etape 6: Traitement du Texte avec RoBERTa

Conversion des textes en representations vectorielles de 768 dimensions.


In [ ]:
# Initialisation du processeur de texte
print("Initialisation du processeur de texte RoBERTa...")
processor = TextProcessor()

# Transformation des donnees d'entrainement
print("\nTransformation des donnees d'entrainement...")
X_train_transformer = processor.text_to_tensor(text_X_train, batch_size=16)

# Transformation des donnees de test
print("Transformation des donnees de test...")
X_test_transformer = processor.text_to_tensor(text_X_test, batch_size=16)

print(f"\nShape des features d'entrainement: {X_train_transformer.shape}")
print(f"Shape des features de test: {X_test_transformer.shape}")


## Etape 7: Entrainement du Modele XGBoost

Recherche des meilleurs hyperparametres avec GridSearchCV.


In [ ]:
# Entrainement du modele XGBoost avec support FHE
print("Entrainement du modele XGBoost...")

model = XGBClassifier()
parameters = {
    "n_bits": [2, 3],  # Bits de quantification FHE
    "max_depth": [1, 2],
    "n_estimators": [30, 50],
    "n_jobs": [-1],
}

print("Recherche des meilleurs hyperparametres...")
grid_search = GridSearchCV(
    model, 
    parameters, 
    cv=3,
    n_jobs=1, 
    scoring="accuracy",
    verbose=1
)

grid_search.fit(X_train_transformer, y_train)

print(f"\nMeilleur score (CV): {grid_search.best_score_:.4f}")
print(f"Meilleurs parametres: {grid_search.best_params_}")

best_model = grid_search.best_estimator_


## Etape 8: Evaluation du Modele

Evaluation sur le jeu de test (20% des donnees).


In [ ]:
# Evaluation sur le jeu de test
print("Evaluation sur le jeu de test...")
y_pred = best_model.predict(X_test_transformer)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nPrecision sur le test: {accuracy:.4f}")
print("\nRapport de classification:")
print(classification_report(y_test, y_pred))
print("\nMatrice de confusion:")
print(confusion_matrix(y_test, y_pred))


## Etape 9: Compilation du Modele pour FHE (CRITIQUE)

Cette etape transforme le modele standard en circuit cryptographique FHE executable.


In [ ]:
# ETAPE CRITIQUE: Compiler le modele pour FHE
# Cette etape transforme le modele en circuit cryptographique
# 
# CE QUI SE PASSE:
# 1. QUANTIFICATION: Float → entiers
# 2. COMPILATION: Modele → circuit FHE
# 3. GENERATION DES CLES: Cle secrete, cle publique, cles d'evaluation
#
# X_train_transformer[:100] est utilise comme echantillon pour calibrer
# la quantification (determiner les bornes min/max pour chaque feature)

print("Compilation du modele pour FHE...")
start = time.perf_counter()
best_model = compile_model(best_model, X_train_transformer[:100])
end = time.perf_counter()
print(f"Temps de compilation: {end - start:.2f} secondes")

# Sauvegarde du modele compile avec ses cles cryptographiques
# Le modele est sauvegarde dans models/sentiment_fhe_model/
# avec le circuit FHE et toutes les cles necessaires
save_model(best_model, "sentiment_fhe_model")

# Sauvegarde du processeur de texte
os.makedirs("models", exist_ok=True)
with open("models/text_processor.pkl", "wb") as f:
    pickle.dump(processor, f)

print("\nModele et processeur sauvegardes avec succes!")


## Etape 10: Test de Prediction FHE

Test du flux complet: Chiffrement → Prediction FHE → Dechiffrement


In [ ]:
# Test de prediction FHE sur un texte d'exemple
test_text = ["Ce produit est incroyable! Je l'adore!"]
X_test = processor.text_to_tensor(test_text)

print("Test de prediction FHE...")
print(f"Texte d'entree: {test_text[0]}")
print("\n" + "="*60)
print("PROCESSUS DE PREDICTION FHE:")
print("="*60)

# ETAPE 1: Texte → Vecteur (EN CLAIR)
print("\nETAPE 1: Texte → Vecteur (EN CLAIR)")
print(f"Shape: {X_test.shape}")
print(f"Type: {X_test.dtype}")
print(f"Premieres 5 valeurs: {X_test[0][:5]}")

# ETAPES 2-5: Prediction FHE (automatique)
print("\nETAPES 2-5: Prediction FHE (automatique)")
print("  - Quantification: Float → Integer")
print("  - Chiffrement: Clair → Ciphertext")
print("  - Calcul FHE: Operations sur donnees chiffrees")
print("  - Dechiffrement: Ciphertext → Clair")

start = time.perf_counter()
# execute_in_fhe=True declenche tout le processus FHE
# 
# CE QUI SE PASSE AUTOMATIQUEMENT DANS best_model.predict():
# 
# 1. QUANTIFICATION (automatique):
#    - X (float32) → X_quant (entiers)
#    - Conversion selon les parametres de quantification du modele
#
# 2. CHIFFREMENT (automatique):
#    - X_quant → X_chiffre (ciphertext)
#    - Utilise la cle publique generee lors de la compilation
#    - Les donnees deviennent illisibles (chiffrees)
#
# 3. PREDICTION FHE (sur donnees chiffrees):
#    - Le circuit FHE traite X_chiffre
#    - Toutes les operations (additions, multiplications) sont faites sur ciphertext
#    - Le modele ne voit jamais les donnees en clair
#    - Resultat: Prediction chiffree (ciphertext)
#
# 4. DECHIFFREMENT (automatique):
#    - Prediction chiffree → Prediction en clair
#    - Utilise la cle secrete (privee)
#    - Resultat final: 0 (negatif) ou 1 (positif)
#
prediction = best_model.predict(X_test, execute_in_fhe=True)
proba = best_model.predict_proba(X_test, execute_in_fhe=True)
end = time.perf_counter()

sentiment_labels = ["Negatif", "Positif"]
sentiment = sentiment_labels[int(prediction[0])]

print(f"\nRESULTAT:")
print(f"  Sentiment: {sentiment}")
print(f"  Prediction: {int(prediction[0])} (0=Negatif, 1=Positif)")
print(f"  Probabilites: {proba}")
print(f"  Temps de traitement FHE: {end - start:.4f} secondes")
print("\n" + "="*60)
print("SUCCES: Prediction terminee sur donnees chiffrees!")
print("="*60)


## Etape 11: Creation du Fichier Zip pour Telechargement

Creation d'un fichier zip contenant le modele FHE et le processeur de texte.


In [ ]:
# Creation du fichier zip pour telechargement
import zipfile

print("Creation du fichier zip pour telechargement...")

with zipfile.ZipFile('fhe_model.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Ajout du repertoire du modele FHE
    model_dir = Path('models/sentiment_fhe_model')
    if model_dir.exists():
        for file in model_dir.rglob('*'):
            if file.is_file():
                zipf.write(file, file.relative_to('models'))
    
    # Ajout du processeur de texte
    proc_file = Path('models/text_processor.pkl')
    if proc_file.exists():
        zipf.write(proc_file, 'text_processor.pkl')

print("Fichier zip cree: fhe_model.zip")
print("\nPour telecharger dans Colab, executez la cellule suivante!")


## Etape 12: Telechargement du Modele (Colab)

Telechargez le fichier zip contenant le modele FHE et le processeur de texte.


In [ ]:
# Telechargement des fichiers du modele
from google.colab import files
files.download('fhe_model.zip')
print("Modele telecharge! Extrayez-le dans votre repertoire local 'models/'.")
print("\nStructure attendue apres extraction:")
print("models/")
print("  sentiment_fhe_model/")
print("    (fichiers du modele FHE)")
print("  text_processor.pkl")


## Etape 13: Demonstration Interactive (Optionnel)

Interface Gradio pour tester le modele FHE de maniere interactive.


In [ ]:
# Chargement du modele pour la demonstration
model_api = load_model("sentiment_fhe_model")

def analyze_sentiment(text: str, use_fhe: bool = True):
    """Analyse le sentiment avec des informations etape par etape."""
    if not text or len(text.strip()) == 0:
        return "Veuillez entrer un texte a analyser."
    
    try:
        # ETAPE 1: Texte → Vecteur (EN CLAIR)
        X = processor.text_to_tensor([text])
        
        result = f"ETAPE 1: Texte → Vecteur (EN CLAIR)\n"
        result += f"Shape: {X.shape}, Type: {X.dtype}\n"
        result += f"Premieres 5 valeurs: {X[0][:5]}\n\n"
        
        if use_fhe:
            result += "ETAPES 2-5: Processus FHE (Automatique)\n"
            result += "  - Quantification: Float → Integer\n"
            result += "  - Chiffrement: Clair → Ciphertext\n"
            result += "  - Calcul FHE: Sur donnees chiffrees\n"
            result += "  - Dechiffrement: Ciphertext → Clair\n\n"
            
            start = time.time()
            prediction = model_api.predict(X, execute_in_fhe=True)
            elapsed = time.time() - start
            
            sentiment_labels = ["Negatif", "Positif"]
            sentiment = sentiment_labels[int(prediction[0])]
            
            result += f"RESULTAT:\n"
            result += f"  Sentiment: {sentiment}\n"
            result += f"  Prediction: {int(prediction[0])}\n"
            result += f"  Temps: {elapsed:.4f}s"
        else:
            prediction = model_api.predict(X, execute_in_fhe=False)
            sentiment_labels = ["Negatif", "Positif"]
            sentiment = sentiment_labels[int(prediction[0])]
            result += f"Resultat (Clair): {sentiment}"
        
        return result
    except Exception as e:
        return f"Erreur: {str(e)}"

# Creation de l'interface Gradio
demo = gr.Interface(
    fn=analyze_sentiment,
    inputs=[
        gr.Textbox(label="Entrez votre texte", placeholder="Ex: Ce produit est incroyable!"),
        gr.Checkbox(label="Utiliser FHE", value=True)
    ],
    outputs=gr.Textbox(label="Resultat", lines=10),
    title="Analyse de Sentiment avec FHE",
    description="Analysez le sentiment tout en gardant les donnees chiffrees pendant le traitement."
)

demo.launch(share=True)
